In [1]:
import pandas as pd
import itertools
import os

### Results for Group Separation Experiments

In [2]:
EXPERIMENT_RESULTS_PATH = 'C:/Users/Tobias/Eigene Dokumente/Research/similaritybench/experiments/results'
COMPARISON_TYPE_DICT = {
    "label_test": "group_separation",
    "layer_test": "monotonicity",
    "augmentation_test": "group_separation",
    "shortcut_test": "group_separation"
}

EXPERIMENTS = list(COMPARISON_TYPE_DICT.keys())
DATASETS = ["cora", "flickr", "ogbn-arxiv"]
ARCHITECTURES = ["GCN", "GraphSAGE", "GAT"]

def FULL_DF_FILE_NAME(experiment, comparison_type, dataset, groups=5):
    if reduced:
        return f"{experiment}_{comparison_type}_{dataset}_{groups}groups_full.csv"
    return f"{experiment}_{comparison_type}_{dataset}_full.csv"

In [3]:
def get_pivot_table(experiment, dataset):
    path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
    
    df = pd.read_csv(path)
    data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]]
    return data.pivot(index="similarity_measure", columns=["representation_dataset", "architecture", "quality_measure"], values="value")

In [4]:
def get_agg_pivot_table(experiment, datasets, groups=5):
    dfs = []
    for dataset in datasets:
        path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset, groups))
        df = pd.read_csv(path)
        data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]]
        dfs.append(data.iloc[:])
    df_cc = pd.concat(dfs, axis=0)
    return df_cc.pivot_table(index="similarity_measure", columns=["representation_dataset", "architecture", "quality_measure"], values="value", aggfunc="mean")

In [5]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.277452         0.4470  0.296727   
CKA                                        0.274583         0.4866  0.273912   
ConcentricityDifference                    0.183074         0.3736  0.136865   
DistanceCorrelation                        0.274829         0.4788  0.281172   
EigenspaceOverlapScore                     0.281568         0.3820  0.272522   
Gulp                                       0.281360         0.3846  0.276770   
HardCorrelationMatch                       0.270700         0.4956  0.271883   
JaccardSimilarity                          0.277164         0.4368  0.271703   
LinearRegression                           0.277083         0.4362  0.282610   
MagnitudeDifference                        0.165867         0.3734  0.114680   
OrthogonalAngularShapeMetricCentered       0.274632         0.4744  0.272782   
OrthogonalProcrustesCenteredAndNormalized  0.274632         0.4744  0.272782   
PermutationProcrustes                      0.278210         0.4044  0.288590   
ProcrustesSizeAndShapeDistance             0.280538         0.4092  0.292431   
RSA                                        0.275556         0.4714  0.291926   
RankSimilarity                             0.297269         0.3836  0.274041   
SVCCA                                      0.228907         0.4420  0.178089   
SecondOrderCosineSimilarity                0.341498         0.2262  0.340875   
SoftCorrelationMatch                       0.271011         0.5016  0.272196   
UniformityDifference                       0.230988         0.3242  0.186015   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.3768  0.281042   
CKA                                               0.4736  0.274771   
ConcentricityDifference                           0.4066  0.162598   
DistanceCorrelation                               0.4270  0.289221   
EigenspaceOverlapScore                            0.4294  0.273769   
Gulp                                              0.4158  0.274428   
HardCorrelationMatch                              0.4790  0.273310   
JaccardSimilarity                                 0.4868  0.273186   
LinearRegression                                  0.4044  0.287879   
MagnitudeDifference                               0.4668  0.176162   
OrthogonalAngularShapeMetricCentered              0.4824  0.275927   
OrthogonalProcrustesCenteredAndNormalized         0.4824  0.275927   
PermutationProcrustes                             0.3814  0.303784   
ProcrustesSizeAndShapeDistance                    0.3568  0.292976   
RSA                                               0.3898  0.272388   
RankSimilarity                                    0.4436  0.269704   
SVCCA                                             0.4736  0.268748   
SecondOrderCosineSimilarity                       0.2600  0.369840   
SoftCorrelationMatch                              0.4822  0.273663   
UniformityDifference                              0.3586  0.275115   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.4118  0.141449   
CKA                                             

In [6]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"], reduced=True)

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.419821       0.456667  0.494788   
CKA                                        0.419536       0.470000  0.416974   
ConcentricityDifference                    0.395347       0.266000  0.244622   
DistanceCorrelation                        0.418925       0.468667  0.426573   
EigenspaceOverlapScore                     0.476001       0.256000  0.456515   
Gulp                                       0.475085       0.255333  0.476787   
HardCorrelationMatch                       0.419056       0.468000  0.415229   
JaccardSimilarity                          0.428521       0.422667  0.418521   
LinearRegression                           0.446764       0.390667  0.483585   
MagnitudeDifference                        0.356165       0.279333  0.199090   
OrthogonalAngularShapeMetricCentered       0.418191       0.472000  0.415585   
OrthogonalProcrustesCenteredAndNormalized  0.418191       0.472000  0.415585   
PermutationProcrustes                      0.418723       0.460667  0.466242   
ProcrustesSizeAndShapeDistance             0.420631       0.449333  0.458124   
RSA                                        0.415736       0.480000  0.499248   
RankSimilarity                             0.481647       0.292000  0.464486   
SVCCA                                      0.371331       0.426667  0.283996   
SecondOrderCosineSimilarity                0.602158       0.140000  0.658267   
SoftCorrelationMatch                       0.418062       0.478667  0.416076   
UniformityDifference                       0.420090       0.289333  0.359575   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.355333  0.427998   
CKA                                             0.468000  0.421428   
ConcentricityDifference                         0.366667  0.365555   
DistanceCorrelation                             0.434667  0.483020   
EigenspaceOverlapScore                          0.313333  0.425640   
Gulp                                            0.281333  0.426779   
HardCorrelationMatch                            0.478000  0.422712   
JaccardSimilarity                               0.472000  0.417107   
LinearRegression                                0.304667  0.444090   
MagnitudeDifference                             0.485333  0.269929   
OrthogonalAngularShapeMetricCentered            0.474667  0.421071   
OrthogonalProcrustesCenteredAndNormalized       0.474667  0.421071   
PermutationProcrustes                           0.344667  0.501982   
ProcrustesSizeAndShapeDistance                  0.328667  0.483543   
RSA                                             0.362000  0.414441   
RankSimilarity                                  0.325333  0.417094   
SVCCA                                           0.522000  0.370423   
SecondOrderCosineSimilarity                     0.159333  0.668338   
SoftCorrelationMatch                            0.473333  0.422280   
UniformityDifference                            0.260000  0.507604   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.413333  0.285138   
CKA                                             

In [7]:
get_agg_pivot_table("layer_test", ["cora", "flickr", "ogbn-arxiv"])

representation_dataset                           cora                 \
architecture                                      GAT                  
quality_measure                           correlation violation_rate   
similarity_measure                                                     
AlignedCosineSimilarity                      0.810646       0.100000   
CKA                                          0.997449       0.004762   
ConcentricityDifference                      0.640306       0.122619   
DistanceCorrelation                          0.998469       0.003571   
EigenspaceOverlapScore                       1.000000       0.000000   
Gulp                                         0.719048       0.091667   
HardCorrelationMatch                         0.905782       0.066667   
JaccardSimilarity                            0.998299       0.005952   
LinearRegression                             0.878776       0.045238   
MagnitudeDifference                          0.930306       0.083333   
OrthogonalAngularShapeMetricCentered         0.999660       0.001190   
OrthogonalProcrustesCenteredAndNormalized    0.999660       0.001190   
PermutationProcrustes                        0.597619       0.158333   
ProcrustesSizeAndShapeDistance               0.444728       0.301190   
RSA                                          0.797041       0.122619   
RankSimilarity                               0.995578       0.013095   
SVCCA                                        0.919762       0.089286   
SecondOrderCosineSimilarity                  0.916599       0.085714   
SoftCorrelationMatch                         0.505646       0.223810   
UniformityDifference                         0.478401       0.297619   

representation_dataset                                                \
architecture                                      GCN                  
quality_measure                           correlation violation_rate   
similarity_measure                                                     
AlignedCosineSimilarity                      0.947177       0.054762   
CKA                                          0.972143       0.032143   
ConcentricityDifference                      0.717585       0.214286   
DistanceCorrelation                          0.988639       0.005952   
EigenspaceOverlapScore                       1.000000       0.000000   
Gulp                                         0.930612       0.028571   
HardCorrelationMatch                         0.914456       0.047619   
JaccardSimilarity                            1.000000       0.000000   
LinearRegression                             0.996803       0.007143   
MagnitudeDifference                          0.345000       0.409524   
OrthogonalAngularShapeMetricCentered         0.991837       0.004762   
OrthogonalProcrustesCenteredAndNormalized    0.991837       0.004762   
PermutationProcrustes                        0.926293       0.052381   
ProcrustesSizeAndShapeDistance               0.998469       0.004762   
RSA                                          0.865680       0.098810   
RankSimilarity                               0.997891       0.003571   
SVCCA                                        0.555340       0.303571   
SecondOrderCosineSimilarity                  0.996939       0.008333   
SoftCorrelationMatch                         0.967279       0.032143   
UniformityDifference                         0.704592       0.197619   

representation_dataset                                                \
architecture                                GraphSAGE                  
quality_measure                           correlation violation_rate   
similarity_measure                                                     
AlignedCosineSimilarity                      0.888639       0.090476   
CKA                                          0.982687       0.030952   
ConcentricityDifference                      0.229694       0.440476   
DistanceCorrelation                   

In [5]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.116701         0.4952  0.144859   
CKA                                        0.226215         0.4034  0.134421   
ConcentricityDifference                    0.095087         0.5468  0.151224   
DistanceCorrelation                        0.221667         0.4104  0.142364   
EigenspaceOverlapScore                     0.084755         0.5932  0.107221   
Gulp                                       0.088260         0.5852  0.103083   
HardCorrelationMatch                       0.107221         0.4652  0.095984   
JaccardSimilarity                          0.172286         0.3460  0.144966   
LinearRegression                           0.157208         0.4180  0.139340   
MagnitudeDifference                        0.091114         0.5432  0.137015   
OrthogonalAngularShapeMetricCentered       0.158798         0.4274  0.131351   
OrthogonalProcrustesCenteredAndNormalized  0.158798         0.4274  0.131351   
PermutationProcrustes                      0.100290         0.5200  0.099663   
ProcrustesSizeAndShapeDistance             0.149436         0.4368  0.123857   
RSA                                        0.229283         0.3280  0.139719   
RankSimilarity                             0.164533         0.3604  0.146545   
SVCCA                                      0.274905         0.3838  0.118372   
SecondOrderCosineSimilarity                0.200859         0.3686  0.159845   
SoftCorrelationMatch                       0.107233         0.4672  0.098392   
UniformityDifference                       0.118079         0.5116  0.095482   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.4022  0.210714   
CKA                                               0.4016  0.303313   
ConcentricityDifference                           0.5348  0.094332   
DistanceCorrelation                               0.3862  0.370273   
EigenspaceOverlapScore                            0.4662  0.187584   
Gulp                                              0.4828  0.187679   
HardCorrelationMatch                              0.5066  0.180108   
JaccardSimilarity                                 0.3994  0.346899   
LinearRegression                                  0.4376  0.363778   
MagnitudeDifference                               0.5326  0.090777   
OrthogonalAngularShapeMetricCentered              0.4062  0.345330   
OrthogonalProcrustesCenteredAndNormalized         0.4062  0.345330   
PermutationProcrustes                             0.4774  0.120771   
ProcrustesSizeAndShapeDistance                    0.4170  0.341849   
RSA                                               0.3884  0.161189   
RankSimilarity                                    0.4218  0.341041   
SVCCA                                             0.5128  0.170292   
SecondOrderCosineSimilarity                       0.3744  0.441520   
SoftCorrelationMatch                              0.5004  0.195404   
UniformityDifference                              0.5270  0.282425   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.2494  0.359970   
CKA                                             

In [6]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"], reduced=True)

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.302898       0.411333  0.318647   
CKA                                        0.462660       0.364000  0.272784   
ConcentricityDifference                    0.204222       0.502000  0.182471   
DistanceCorrelation                        0.459166       0.378000  0.283945   
EigenspaceOverlapScore                     0.163367       0.575333  0.216938   
Gulp                                       0.172056       0.560667  0.197004   
HardCorrelationMatch                       0.219276       0.418667  0.200268   
JaccardSimilarity                          0.379688       0.286000  0.305227   
LinearRegression                           0.333709       0.376000  0.299139   
MagnitudeDifference                        0.187117       0.511333  0.267258   
OrthogonalAngularShapeMetricCentered       0.365977       0.380000  0.274114   
OrthogonalProcrustesCenteredAndNormalized  0.365977       0.380000  0.274114   
PermutationProcrustes                      0.224682       0.480667  0.207401   
ProcrustesSizeAndShapeDistance             0.370079       0.396000  0.247264   
RSA                                        0.578102       0.225333  0.298501   
RankSimilarity                             0.361924       0.276667  0.331721   
SVCCA                                      0.571318       0.301333  0.191337   
SecondOrderCosineSimilarity                0.471775       0.311333  0.342682   
SoftCorrelationMatch                       0.217770       0.421333  0.209331   
UniformityDifference                       0.187576       0.525333  0.191554   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.305333  0.598752   
CKA                                             0.338667  0.776866   
ConcentricityDifference                         0.532000  0.168895   
DistanceCorrelation                             0.312000  0.819571   
EigenspaceOverlapScore                          0.426000  0.495157   
Gulp                                            0.474667  0.495904   
HardCorrelationMatch                            0.441333  0.347125   
JaccardSimilarity                               0.303333  0.782344   
LinearRegression                                0.383333  0.741737   
MagnitudeDifference                             0.519333  0.151357   
OrthogonalAngularShapeMetricCentered            0.325333  0.794205   
OrthogonalProcrustesCenteredAndNormalized       0.325333  0.794205   
PermutationProcrustes                           0.412000  0.267543   
ProcrustesSizeAndShapeDistance                  0.355333  0.812004   
RSA                                             0.298000  0.432189   
RankSimilarity                                  0.345333  0.644560   
SVCCA                                           0.538000  0.357298   
SecondOrderCosineSimilarity                     0.256000  0.906817   
SoftCorrelationMatch                            0.430000  0.404097   
UniformityDifference                            0.482000  0.636742   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.118667  0.479826   
CKA                                             

In [9]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.293559         0.1662  0.158459   
CKA                                        0.390306         0.1230  0.223380   
ConcentricityDifference                    0.228318         0.2942  0.106856   
DistanceCorrelation                        0.409174         0.1104  0.289524   
EigenspaceOverlapScore                     0.118466         0.3920  0.132337   
Gulp                                       0.120187         0.3842  0.132054   
HardCorrelationMatch                       0.125550         0.3862  0.131358   
JaccardSimilarity                          0.425655         0.0876  0.295210   
LinearRegression                           0.317356         0.1230  0.341739   
MagnitudeDifference                        0.346935         0.2634  0.128563   
OrthogonalAngularShapeMetricCentered       0.309252         0.1476  0.232918   
OrthogonalProcrustesCenteredAndNormalized  0.309252         0.1476  0.232918   
PermutationProcrustes                      0.210557         0.5340  0.218047   
ProcrustesSizeAndShapeDistance             0.361892         0.1702  0.412195   
RSA                                        0.355900         0.1990  0.125867   
RankSimilarity                             0.395225         0.1090  0.279279   
SVCCA                                      0.120114         0.5238  0.156106   
SecondOrderCosineSimilarity                0.475680         0.0796  0.341902   
SoftCorrelationMatch                       0.130398         0.3744  0.148219   
UniformityDifference                       0.842913         0.0164  0.311111   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.3958  0.292466   
CKA                                               0.2852  0.294391   
ConcentricityDifference                           0.4880  0.287110   
DistanceCorrelation                               0.2332  0.326323   
EigenspaceOverlapScore                            0.3552  0.195475   
Gulp                                              0.3620  0.203648   
HardCorrelationMatch                              0.3606  0.178539   
JaccardSimilarity                                 0.2038  0.343669   
LinearRegression                                  0.1158  0.350219   
MagnitudeDifference                               0.4412  0.201762   
OrthogonalAngularShapeMetricCentered              0.2368  0.314976   
OrthogonalProcrustesCenteredAndNormalized         0.2368  0.314976   
PermutationProcrustes                             0.3534  0.198017   
ProcrustesSizeAndShapeDistance                    0.2470  0.310849   
RSA                                               0.5034  0.167792   
RankSimilarity                                    0.3496  0.323344   
SVCCA                                             0.5404  0.124082   
SecondOrderCosineSimilarity                       0.2080  0.349285   
SoftCorrelationMatch                              0.3292  0.197792   
UniformityDifference                              0.2584  0.296352   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                           0.3202  0.436004   
CKA                                             

In [10]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"], reduced=True)

representation_dataset                         cora                           \
architecture                                    GAT                      GCN   
quality_measure                               AUPRC violation_rate     AUPRC   
similarity_measure                                                             
AlignedCosineSimilarity                    0.786291       0.034667  0.359539   
CKA                                        0.692267       0.093333  0.526463   
ConcentricityDifference                    0.553624       0.165333  0.175102   
DistanceCorrelation                        0.719894       0.078667  0.597550   
EigenspaceOverlapScore                     0.253580       0.328000  0.285511   
Gulp                                       0.258547       0.317333  0.293003   
HardCorrelationMatch                       0.255907       0.312000  0.308416   
JaccardSimilarity                          0.768198       0.036667  0.619864   
LinearRegression                           0.609928       0.099333  0.644813   
MagnitudeDifference                        0.535091       0.172667  0.196374   
OrthogonalAngularShapeMetricCentered       0.598313       0.120000  0.569424   
OrthogonalProcrustesCenteredAndNormalized  0.598313       0.120000  0.569424   
PermutationProcrustes                      0.332129       0.500000  0.509570   
ProcrustesSizeAndShapeDistance             0.741770       0.113333  0.698673   
RSA                                        0.724276       0.090000  0.245898   
RankSimilarity                             0.839487       0.038667  0.446553   
SVCCA                                      0.245228       0.541333  0.266658   
SecondOrderCosineSimilarity                0.818636       0.035333  0.686595   
SoftCorrelationMatch                       0.263044       0.311333  0.399426   
UniformityDifference                       1.000000       0.000000  0.673593   

representation_dataset                                              \
architecture                                             GraphSAGE   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.314667  0.547878   
CKA                                             0.190667  0.652736   
ConcentricityDifference                         0.508000  0.487404   
DistanceCorrelation                             0.135333  0.698952   
EigenspaceOverlapScore                          0.256000  0.415019   
Gulp                                            0.252000  0.415199   
HardCorrelationMatch                            0.277333  0.397374   
JaccardSimilarity                               0.142000  0.762113   
LinearRegression                                0.082000  0.772596   
MagnitudeDifference                             0.468000  0.489759   
OrthogonalAngularShapeMetricCentered            0.134000  0.705962   
OrthogonalProcrustesCenteredAndNormalized       0.134000  0.705962   
PermutationProcrustes                           0.283333  0.387604   
ProcrustesSizeAndShapeDistance                  0.147333  0.646733   
RSA                                             0.471333  0.358463   
RankSimilarity                                  0.316000  0.598879   
SVCCA                                           0.506000  0.289264   
SecondOrderCosineSimilarity                     0.144000  0.779901   
SoftCorrelationMatch                            0.236667  0.423943   
UniformityDifference                            0.101333  0.503574   

representation_dataset                                      flickr  \
architecture                                                   GAT   
quality_measure                           violation_rate     AUPRC   
similarity_measure                                                   
AlignedCosineSimilarity                         0.188000  0.524542   
CKA                                             

### Results of Output Correlations

In [11]:
def get_output_correlation_table(experiment, dataset):
    
    path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, "output_correlation", dataset))
    df = pd.read_csv(path)
    data = df.loc[:, ["similarity_measure", "functional_similarity_measure", "quality_measure", "corr", "architecture"]]
    
    return data.pivot_table(index="similarity_measure", columns=["architecture", "functional_similarity_measure", "quality_measure"], values="corr", aggfunc="mean")

In [12]:
get_output_correlation_table("label_test", "cora")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.509585 -0.916551 -0.670872   
CKA                                          -0.473548 -0.900428 -0.617185   
ConcentricityDifference                      -0.105900 -0.204555 -0.163648   
DistanceCorrelation                          -0.468591 -0.923049 -0.611785   
EigenspaceOverlapScore                       -0.272375 -0.448225 -0.397818   
Gulp                                          0.261033  0.439794  0.382707   
HardCorrelationMatch                         -0.488105 -0.919895 -0.629989   
JaccardSimilarity                            -0.534502 -0.764992 -0.697462   
LinearRegression                             -0.438986 -0.838750 -0.598718   
MagnitudeDifference                          -0.125370 -0.230291 -0.185317   
OrthogonalAngularShapeMetricCentered          0.463143  0.899142  0.613993   
OrthogonalProcrustesCenteredAndNormalized     0.463143  0.891554  0.613993   
PermutationProcrustes                         0.378119  0.838350  0.523389   
ProcrustesSizeAndShapeDistance                0.440860  0.868290  0.596751   
RSA                                          -0.493777 -0.892949 -0.637358   
RankSimilarity                               -0.318326 -0.666559 -0.452853   
SVCCA                                        -0.311806 -0.681743 -0.452274   
SecondOrderCosineSimilarity                  -0.311806 -0.702177 -0.429537   
SoftCorrelationMatch                         -0.482434 -0.919467 -0.628452   
UniformityDifference                         -0.146492 -0.220471 -0.212037   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.535206 -0.884646 -0.688909   
CKA                                        -0.448027 -0.857739 -0.601184   
ConcentricityDifference                    -0.077280 -0.164305 -0.122245   
DistanceCorrelation                        -0.453110 -0.880671 -0.603781   
EigenspaceOverlapScore                     -0.301583 -0.478920 -0.436154   
Gulp                                        0.289810  0.469977  0.421743   
HardCorrelationMatch                       -0.460468 -0.876687 -0.612355   
JaccardSimilarity                          -0.535964 -0.728527 -0.697987   
LinearRegression                           -0.424974 -0.785515 -0.577432   
MagnitudeDifference                        -0.098729 -0.198162 -0.149061   
OrthogonalAngularShapeMetricCentered        0.483344  0.861048  0.626745   
OrthogonalProcrustesCenteredAndNormalized   0.483344  0.853370  0.626745   
PermutationProcrustes                       0.361828  0.788281  0.505443   
ProcrustesSizeAndShapeDistance              0.442230  0.826328  0.599229   
RSA                                        -0.477146 -0.850965 -0.627669   
RankSimilarity                             -0.346087 -0.653213 -0.491507   
SVCCA                                      -0.305552 -0.649424 -0.443954   
SecondOrderCosineSimilarity                -0.380468 -0.718435 -0.514335   
SoftCorrelationMatch                       -0.451282 -0.873729 -0.605599   
UniformityDifference                       -0.142965 -0.218451 -0.202132   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [13]:
get_output_correlation_table("label_test", "flickr")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.075699 -0.117805 -0.106484   
CKA                                          -0.007603  0.007734 -0.005532   
ConcentricityDifference                      -0.037214 -0.089210 -0.060974   
DistanceCorrelation                          -0.052733 -0.004947 -0.065580   
EigenspaceOverlapScore                        0.030971  0.057493  0.041775   
Gulp                                         -0.026957 -0.059360 -0.045520   
HardCorrelationMatch                         -0.183752 -0.197044 -0.269195   
JaccardSimilarity                            -0.145534 -0.081065 -0.221831   
LinearRegression                              0.006494  0.026687  0.010612   
MagnitudeDifference                          -0.051663 -0.092329 -0.076602   
OrthogonalAngularShapeMetricCentered         -0.001940 -0.032646 -0.007450   
OrthogonalProcrustesCenteredAndNormalized    -0.001940 -0.028037 -0.007450   
PermutationProcrustes                        -0.058530 -0.102082 -0.082727   
ProcrustesSizeAndShapeDistance               -0.057326 -0.107595 -0.079999   
RSA                                          -0.092645 -0.114242 -0.141826   
RankSimilarity                               -0.202482 -0.124890 -0.300430   
SVCCA                                        -0.004794  0.038270 -0.001409   
SecondOrderCosineSimilarity                  -0.077171 -0.110447 -0.114552   
SoftCorrelationMatch                         -0.096570 -0.103582 -0.142129   
UniformityDifference                         -0.017102 -0.038399 -0.019413   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.082096 -0.083086 -0.119403   
CKA                                        -0.174493 -0.244961 -0.262611   
ConcentricityDifference                     0.093556  0.078170  0.141156   
DistanceCorrelation                        -0.194872 -0.256748 -0.291142   
EigenspaceOverlapScore                     -0.011237  0.020557 -0.016646   
Gulp                                        0.251996  0.356353  0.362350   
HardCorrelationMatch                       -0.334181 -0.406197 -0.472265   
JaccardSimilarity                          -0.238261 -0.293316 -0.350141   
LinearRegression                           -0.145586 -0.182693 -0.217637   
MagnitudeDifference                        -0.110100 -0.166321 -0.164365   
OrthogonalAngularShapeMetricCentered        0.172575  0.239208  0.259006   
OrthogonalProcrustesCenteredAndNormalized   0.172575  0.242265  0.259006   
PermutationProcrustes                      -0.117458 -0.184553 -0.176886   
ProcrustesSizeAndShapeDistance             -0.078172 -0.139745 -0.118460   
RSA                                        -0.228763 -0.357973 -0.339744   
RankSimilarity                             -0.255162 -0.289039 -0.372154   
SVCCA                                      -0.180424 -0.234544 -0.269874   
SecondOrderCosineSimilarity                -0.230234 -0.351809 -0.339588   
SoftCorrelationMatch                       -0.304526 -0.407611 -0.438640   
UniformityDifference                        0.266444  0.377121  0.394088   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [14]:
get_output_correlation_table("label_test", "ogbn-arxiv")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.650813 -0.834448 -0.862018   
CKA                                          -0.564533 -0.737850 -0.774701   
ConcentricityDifference                       0.073137 -0.025403  0.105955   
DistanceCorrelation                          -0.648444 -0.844284 -0.858611   
EigenspaceOverlapScore                       -0.684700 -0.883540 -0.890176   
Gulp                                          0.683359  0.881338  0.888764   
HardCorrelationMatch                         -0.687471 -0.910824 -0.891366   
JaccardSimilarity                            -0.692925 -0.836132 -0.895709   
LinearRegression                             -0.635418 -0.852553 -0.856289   
MagnitudeDifference                           0.007555 -0.084838  0.009174   
OrthogonalAngularShapeMetricCentered          0.704862  0.893964  0.899717   
OrthogonalProcrustesCenteredAndNormalized     0.704862  0.889897  0.899717   
PermutationProcrustes                         0.123788  0.110715  0.160434   
ProcrustesSizeAndShapeDistance                0.559660  0.689122  0.772994   
RSA                                          -0.627030 -0.943610 -0.858060   
RankSimilarity                               -0.684610 -0.816889 -0.887691   
SVCCA                                        -0.334482 -0.398143 -0.457063   
SecondOrderCosineSimilarity                  -0.669008 -0.845654 -0.874262   
SoftCorrelationMatch                         -0.674596 -0.911672 -0.884540   
UniformityDifference                         -0.100541 -0.227352 -0.152730   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.560401 -0.766566 -0.736649   
CKA                                        -0.721828 -0.881106 -0.902226   
ConcentricityDifference                     0.461984  0.678243  0.639569   
DistanceCorrelation                        -0.629387 -0.803510 -0.822918   
EigenspaceOverlapScore                     -0.323969 -0.506115 -0.474025   
Gulp                                        0.391394  0.553236  0.542778   
HardCorrelationMatch                       -0.505507 -0.644775 -0.669688   
JaccardSimilarity                          -0.470903 -0.633999 -0.675379   
LinearRegression                           -0.277126 -0.408234 -0.425120   
MagnitudeDifference                         0.025909 -0.087285  0.035644   
OrthogonalAngularShapeMetricCentered        0.480045  0.617667  0.668210   
OrthogonalProcrustesCenteredAndNormalized   0.480045  0.616115  0.668210   
PermutationProcrustes                       0.510145  0.734635  0.693780   
ProcrustesSizeAndShapeDistance              0.696187  0.891484  0.870353   
RSA                                        -0.303545 -0.467494 -0.456409   
RankSimilarity                             -0.498194 -0.702282 -0.706885   
SVCCA                                      -0.584615 -0.690085 -0.798069   
SecondOrderCosineSimilarity                -0.579933 -0.817200 -0.790083   
SoftCorrelationMatch                       -0.485842 -0.627711 -0.646385   
UniformityDifference                        0.067157  0.020811  0.093241   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [15]:
get_output_correlation_table("shortcut_test", "cora")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                       0.084346  0.142265  0.113742   
CKA                                          -0.061072 -0.087849 -0.088566   
ConcentricityDifference                      -0.111916 -0.165641 -0.165593   
DistanceCorrelation                          -0.059953 -0.084395 -0.088791   
EigenspaceOverlapScore                       -0.029473 -0.278845 -0.054950   
Gulp                                          0.024370  0.277664  0.047559   
HardCorrelationMatch                         -0.269552 -0.391527 -0.380577   
JaccardSimilarity                            -0.084390 -0.207227 -0.127164   
LinearRegression                             -0.097870 -0.138093 -0.143012   
MagnitudeDifference                          -0.024639 -0.032036 -0.034505   
OrthogonalAngularShapeMetricCentered          0.061788  0.121010  0.093041   
OrthogonalProcrustesCenteredAndNormalized     0.061788  0.122349  0.093041   
PermutationProcrustes                         0.075528  0.013717  0.104779   
ProcrustesSizeAndShapeDistance                0.095625  0.130682  0.138554   
RSA                                          -0.052389 -0.067604 -0.077733   
RankSimilarity                               -0.088284 -0.215118 -0.131557   
SVCCA                                         0.046302  0.078168  0.067533   
SecondOrderCosineSimilarity                  -0.103323 -0.175882 -0.150648   
SoftCorrelationMatch                         -0.272193 -0.379753 -0.385356   
UniformityDifference                          0.042139  0.010974  0.059811   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                     0.035452  0.047240  0.039324   
CKA                                        -0.097748 -0.170038 -0.140643   
ConcentricityDifference                    -0.073534 -0.075348 -0.109595   
DistanceCorrelation                        -0.096499 -0.168136 -0.140059   
EigenspaceOverlapScore                     -0.076477 -0.280811 -0.126497   
Gulp                                        0.074381  0.280183  0.122620   
HardCorrelationMatch                       -0.317547 -0.401708 -0.448451   
JaccardSimilarity                          -0.127982 -0.234752 -0.193739   
LinearRegression                           -0.133601 -0.178021 -0.197584   
MagnitudeDifference                         0.007982  0.049213  0.008663   
OrthogonalAngularShapeMetricCentered        0.113043  0.196388  0.167135   
OrthogonalProcrustesCenteredAndNormalized   0.113043  0.197234  0.167135   
PermutationProcrustes                       0.107648  0.077603  0.152112   
ProcrustesSizeAndShapeDistance              0.137480  0.200273  0.196641   
RSA                                        -0.096678 -0.143220 -0.141387   
RankSimilarity                             -0.126288 -0.237105 -0.189047   
SVCCA                                      -0.005663 -0.015787 -0.007209   
SecondOrderCosineSimilarity                -0.149610 -0.233201 -0.215407   
SoftCorrelationMatch                       -0.297793 -0.379808 -0.422446   
UniformityDifference                        0.063902  0.060877  0.088586   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [16]:
get_output_correlation_table("shortcut_test", "flickr")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.191678 -0.310566 -0.280539   
CKA                                           0.080498  0.112315  0.118377   
ConcentricityDifference                      -0.007983 -0.063764 -0.023280   
DistanceCorrelation                           0.028364  0.087908  0.042061   
EigenspaceOverlapScore                       -0.049191 -0.121116 -0.071692   
Gulp                                          0.316238  0.475386  0.483070   
HardCorrelationMatch                         -0.316416 -0.520786 -0.483867   
JaccardSimilarity                            -0.454489 -0.711051 -0.658291   
LinearRegression                             -0.008879  0.014951 -0.009993   
MagnitudeDifference                          -0.044998 -0.107515 -0.070391   
OrthogonalAngularShapeMetricCentered         -0.056995 -0.072530 -0.085413   
OrthogonalProcrustesCenteredAndNormalized    -0.057040 -0.062989 -0.085437   
PermutationProcrustes                        -0.065023 -0.084768 -0.095289   
ProcrustesSizeAndShapeDistance               -0.066360 -0.090678 -0.100116   
RSA                                          -0.411230 -0.624968 -0.600752   
RankSimilarity                               -0.478750 -0.758043 -0.681949   
SVCCA                                         0.073674  0.139267  0.114277   
SecondOrderCosineSimilarity                  -0.496678 -0.770163 -0.703088   
SoftCorrelationMatch                         -0.269054 -0.451622 -0.410662   
UniformityDifference                          0.226821  0.344143  0.325796   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.177079 -0.288528 -0.261045   
CKA                                         0.051327  0.064812  0.072535   
ConcentricityDifference                     0.018239 -0.005721  0.021270   
DistanceCorrelation                        -0.007581  0.026023 -0.016858   
EigenspaceOverlapScore                     -0.031750 -0.104547 -0.046357   
Gulp                                        0.335072  0.503631  0.506488   
HardCorrelationMatch                       -0.354337 -0.573153 -0.527278   
JaccardSimilarity                          -0.446555 -0.706316 -0.653904   
LinearRegression                           -0.027313 -0.011334 -0.039584   
MagnitudeDifference                        -0.030591 -0.053344 -0.050638   
OrthogonalAngularShapeMetricCentered       -0.027603 -0.026519 -0.039156   
OrthogonalProcrustesCenteredAndNormalized  -0.027559 -0.018511 -0.039135   
PermutationProcrustes                      -0.073712 -0.076844 -0.109899   
ProcrustesSizeAndShapeDistance             -0.071260 -0.077741 -0.109442   
RSA                                        -0.415072 -0.641082 -0.607877   
RankSimilarity                             -0.474470 -0.758286 -0.680945   
SVCCA                                       0.050078  0.096235  0.077916   
SecondOrderCosineSimilarity                -0.478796 -0.762657 -0.690123   
SoftCorrelationMatch                       -0.307915 -0.502936 -0.460193   
UniformityDifference                        0.263946  0.426231  0.386159   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [17]:
get_output_correlation_table("shortcut_test", "ogbn-arxiv")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.494406 -0.705029 -0.687399   
CKA                                          -0.491417 -0.684370 -0.688332   
ConcentricityDifference                       0.102192  0.106593  0.165442   
DistanceCorrelation                          -0.464380 -0.655172 -0.655421   
EigenspaceOverlapScore                       -0.489677 -0.713961 -0.677148   
Gulp                                          0.488205  0.713027  0.675828   
HardCorrelationMatch                         -0.523406 -0.746359 -0.716494   
JaccardSimilarity                            -0.497574 -0.703004 -0.683402   
LinearRegression                             -0.426012 -0.553682 -0.603080   
MagnitudeDifference                           0.245229  0.310460  0.375611   
OrthogonalAngularShapeMetricCentered          0.515866  0.729825  0.707220   
OrthogonalProcrustesCenteredAndNormalized     0.515866  0.730116  0.707220   
PermutationProcrustes                         0.473973  0.701857  0.666791   
ProcrustesSizeAndShapeDistance                0.513591  0.717599  0.707173   
RSA                                          -0.500162 -0.671480 -0.703952   
RankSimilarity                               -0.488874 -0.702583 -0.674339   
SVCCA                                        -0.390675 -0.647684 -0.592148   
SecondOrderCosineSimilarity                  -0.473259 -0.696171 -0.662912   
SoftCorrelationMatch                         -0.510646 -0.739771 -0.702774   
UniformityDifference                          0.373989  0.567169  0.546344   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.606867 -0.829893 -0.804341   
CKA                                        -0.602051 -0.821893 -0.802772   
ConcentricityDifference                     0.161962  0.134704  0.253512   
DistanceCorrelation                        -0.565307 -0.792375 -0.765802   
EigenspaceOverlapScore                     -0.605619 -0.830569 -0.795505   
Gulp                                        0.602809  0.829105  0.792980   
HardCorrelationMatch                       -0.618149 -0.853872 -0.808571   
JaccardSimilarity                          -0.601873 -0.824198 -0.785580   
LinearRegression                           -0.509229 -0.645272 -0.703691   
MagnitudeDifference                         0.322230  0.386105  0.488033   
OrthogonalAngularShapeMetricCentered        0.628673  0.852861  0.818827   
OrthogonalProcrustesCenteredAndNormalized   0.628673  0.852802  0.818827   
PermutationProcrustes                       0.574671  0.836214  0.773221   
ProcrustesSizeAndShapeDistance              0.628673  0.846635  0.822847   
RSA                                        -0.599554 -0.804730 -0.809812   
RankSimilarity                             -0.590145 -0.823869 -0.775871   
SVCCA                                      -0.496990 -0.787678 -0.721060   
SecondOrderCosineSimilarity                -0.578595 -0.829904 -0.770112   
SoftCorrelationMatch                       -0.592821 -0.840047 -0.786507   
UniformityDifference                        0.410123  0.650229  0.603379   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [18]:
get_output_correlation_table("augmentation_test", "cora")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                       0.204598  0.234288  0.250005   
CKA                                           0.159132  0.156891  0.152442   
ConcentricityDifference                      -0.295531 -0.459046 -0.475619   
DistanceCorrelation                           0.159132  0.115880  0.152442   
EigenspaceOverlapScore                        0.068199  0.010870  0.067074   
Gulp                                         -0.113666 -0.003025 -0.079270   
HardCorrelationMatch                         -0.204598 -0.398758 -0.310981   
JaccardSimilarity                            -0.113666 -0.125135 -0.140247   
LinearRegression                              0.298712  0.270406  0.362964   
MagnitudeDifference                          -0.340997 -0.415221 -0.451228   
OrthogonalAngularShapeMetricCentered         -0.250065 -0.419780 -0.274395   
OrthogonalProcrustesCenteredAndNormalized    -0.250065 -0.419709 -0.274395   
PermutationProcrustes                         0.022733  0.207969  0.115856   
ProcrustesSizeAndShapeDistance               -0.431930 -0.526596 -0.542693   
RSA                                           0.386463  0.428138  0.475619   
RankSimilarity                               -0.340997 -0.425012 -0.506107   
SVCCA                                         0.068199  0.078927  0.054879   
SecondOrderCosineSimilarity                   0.159132  0.227005  0.182930   
SoftCorrelationMatch                         -0.250065 -0.393490 -0.384153   
UniformityDifference                         -0.340997 -0.480584 -0.493912   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.022222 -0.085014  0.042424   
CKA                                        -0.333333 -0.450500 -0.454545   
ConcentricityDifference                    -0.066667 -0.028583 -0.090909   
DistanceCorrelation                        -0.333333 -0.476871 -0.454545   
EigenspaceOverlapScore                     -0.244444 -0.528755 -0.284848   
Gulp                                        0.288889  0.600440  0.381818   
HardCorrelationMatch                       -0.288889 -0.382430 -0.357576   
JaccardSimilarity                           0.155556  0.010643  0.200000   
LinearRegression                            0.216295  0.238555  0.307927   
MagnitudeDifference                        -0.288889 -0.193101 -0.369697   
OrthogonalAngularShapeMetricCentered       -0.111111 -0.075314 -0.151515   
OrthogonalProcrustesCenteredAndNormalized  -0.111111 -0.075208 -0.151515   
PermutationProcrustes                      -0.244444  0.172381 -0.248485   
ProcrustesSizeAndShapeDistance             -0.200000 -0.228882 -0.224242   
RSA                                         0.244444  0.184614  0.333333   
RankSimilarity                              0.022222 -0.014346  0.030303   
SVCCA                                      -0.244444 -0.382297 -0.309091   
SecondOrderCosineSimilarity                -0.022222  0.007109 -0.030303   
SoftCorrelationMatch                        0.022222 -0.035404  0.030303   
UniformityDifference                       -0.600000 -0.722462 -0.733333   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

In [19]:
get_output_correlation_table("augmentation_test", "flickr")

architecture                                       GAT                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                           
AlignedCosineSimilarity                      -0.271691 -0.377773 -0.401120   
CKA                                          -0.096064 -0.208925 -0.149080   
ConcentricityDifference                       0.187892  0.240331  0.275380   
DistanceCorrelation                          -0.226960 -0.283499 -0.338588   
EigenspaceOverlapScore                       -0.237351 -0.327099 -0.359101   
Gulp                                          0.307771  0.387008  0.430528   
HardCorrelationMatch                         -0.335734 -0.448182 -0.487163   
JaccardSimilarity                            -0.360129 -0.394317 -0.487002   
LinearRegression                             -0.094842 -0.097058 -0.143719   
MagnitudeDifference                           0.132010  0.160847  0.196387   
OrthogonalAngularShapeMetricCentered          0.107749  0.208772  0.168931   
OrthogonalProcrustesCenteredAndNormalized     0.107749  0.212073  0.168931   
PermutationProcrustes                         0.147709  0.155567  0.223626   
ProcrustesSizeAndShapeDistance                0.159750  0.179644  0.244513   
RSA                                          -0.346215 -0.504610 -0.497145   
RankSimilarity                               -0.371323 -0.465688 -0.516862   
SVCCA                                        -0.091292 -0.173810 -0.143433   
SecondOrderCosineSimilarity                  -0.359594 -0.443477 -0.509396   
SoftCorrelationMatch                         -0.317003 -0.438318 -0.465317   
UniformityDifference                          0.164790  0.283788  0.242868   

architecture                                                              \
functional_similarity_measure                    JSD                       
quality_measure                           kendalltau  pearsonr spearmanr   
similarity_measure                                                         
AlignedCosineSimilarity                    -0.410479 -0.574452 -0.582532   
CKA                                        -0.162542 -0.337277 -0.238281   
ConcentricityDifference                     0.233980  0.255272  0.338894   
DistanceCorrelation                        -0.324326 -0.447187 -0.476038   
EigenspaceOverlapScore                     -0.281695 -0.397515 -0.416631   
Gulp                                        0.478172  0.642937  0.648993   
HardCorrelationMatch                       -0.465596 -0.627715 -0.637693   
JaccardSimilarity                          -0.476433 -0.598656 -0.641084   
LinearRegression                           -0.182150 -0.236980 -0.260674   
MagnitudeDifference                         0.109298  0.130463  0.170528   
OrthogonalAngularShapeMetricCentered        0.183188  0.345559  0.268263   
OrthogonalProcrustesCenteredAndNormalized   0.183188  0.348988  0.268263   
PermutationProcrustes                       0.105507  0.091747  0.164005   
ProcrustesSizeAndShapeDistance              0.135697  0.139003  0.213191   
RSA                                        -0.555496 -0.747460 -0.724892   
RankSimilarity                             -0.558930 -0.712813 -0.736213   
SVCCA                                      -0.163122 -0.297329 -0.240681   
SecondOrderCosineSimilarity                -0.486511 -0.653544 -0.667243   
SoftCorrelationMatch                       -0.503545 -0.684774 -0.692629   
UniformityDifference                        0.345329  0.551603  0.494193   

architecture                                       GCN                      \
functional_similarity_measure             Disagreement                       
quality_measure                             kendalltau  pearsonr spearmanr   
similarity_measure                                                  

### CD Plots

In [34]:
# autorank package needed for creation of CD plots 
from autorank import autorank, plot_stats, create_report, latex_table
from autorank._util import *
from matplotlib import pyplot as plt

In [39]:
def get_figsize(columnwidth, wf=0.5, hf=(5. ** 0.5 - 1.0) / 2.0):
    """ Credit: https://stackoverflow.com/a/31527287
    Parameters:
      - wf [float]:  width fraction in columnwidth units
      - hf [float]:  height fraction in columnwidth units.
                     Set by default to golden ratio.
      - columnwidth [float]: width of the column in latex. Get this from LaTeX
                             using \showthe\columnwidth
    Returns:  [fig_width,fig_height]: that should be given to matplotlib
    """
    fig_width_pt = columnwidth * wf
    inches_per_pt = 1.0 / 72.27  # Convert pt to inch
    fig_width = fig_width_pt * inches_per_pt  # width in inches
    fig_height = fig_width * hf  # height in inches
    return fig_width, fig_height

PLOTS_BASE_WIDTH = 433.62  # pt
PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT = get_figsize(PLOTS_BASE_WIDTH, wf=1.5)

In [28]:
def get_autorank_df(experiments=EXPERIMENTS, datasets=DATASETS, architectures=ARCHITECTURES, quality_measure = "violation_rate"):
    dfs = []
    for experiment in experiments:
        for dataset in datasets:
            path = os.path.join(EXPERIMENT_RESULTS_PATH, FULL_DF_FILE_NAME(experiment, COMPARISON_TYPE_DICT[experiment], dataset))
            df = pd.read_csv(path)
            df["experiment"] = experiment
            data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset", "experiment"]]
            data = data.loc[data.loc[:,"architecture"].isin(architectures)]
            data = data.loc[data.loc[:,"quality_measure"]==quality_measure]
            dfs.append(data)
            
    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="similarity_measure", columns=["experiment", "representation_dataset", "architecture"], values="value", aggfunc="mean")
    return df_res.transpose()

In [38]:
res_df = get_autorank_df()
result = autorank(-res_df, alpha=0.05, verbose=False)
cd_diagram(result, False, None, PLOTS_CD_WIDTH)
fig = plt.gcf()
fig.set_size_inches(PLOTS_CD_WIDTH, PLOTS_CD_HEIGHT)
plt.savefig(f"cd_test.png", bbox_inches="tight")
plt.close()

C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:432: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if abs(sorted_ranks[i] - sorted_ranks[j]) <= critical_difference:
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:511: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  plot_line([(rankpos(sorted_ranks[i]), cline),
C:\Users\Tobias\anaconda3\envs\simba\lib\site-packages\autorank\_util.py:512: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To ac